In [1]:
!pip install emoji
!pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import re
import emoji
import string
from cleantext import clean

In [3]:
#read csv
df1 = pd.read_csv('raw_tweets_p1.csv')
#display(df1)
df2 = pd.read_csv('raw_tweets_p2.csv')
#display(df2)
df3 = pd.read_csv('raw_tweets_p3.csv')
df4 = df1.append(df2, ignore_index=True)
df=df4.append(df3, ignore_index=True)
display(df)


,id,time,tweets,emoji
0,1593507164559056897,2022-11-18 07:31:28,@BLACKPINKSTATS5 Love to see it 👏 the numbers ...,👏
1,1593507163971911682,2022-11-18 07:31:28,@PostmanCarson congratulations John 🎊 👏,👏
2,1593507163506462721,2022-11-18 07:31:28,@Tyranjf @RunningWithTum1 @we_are_runners @Ich...,👏
3,1593507132296527873,2022-11-18 07:31:20,A story book scene.. congrats to both of you! 👏,👏
4,1593507130291769345,2022-11-18 07:31:20,@thatpositivepie 👏👏👏 and good look,👏
...,...,...,...,...
300609,1593520889517297668,2022-11-18 08:26:00,@UKDaddyBear1 Needs my tongue 👅,👅
300610,1593520798521778177,2022-11-18 08:25:39,@LoganFoxx69 Wow nice dick 🍆👅,👅
300611,1593520489259204608,2022-11-18 08:24:25,Would you 💋 or 👅?\n\nbokep indo viral mesum li...,👅
300612,1593520348082970624,2022-11-18 08:23:51,@victoriacake3x Yeaaaah👅👅,👅


In [4]:
temp_data = df.groupby("emoji").size().sort_values(ascending = False)
# temp_list=df.index[df['emoji'] == '👅'].tolist()
# print(len(temp_list))
temp_data

emoji
💛        6082
💗        5000
💘        5000
💚        5000
💜        5000
         ... 
💋        5000
💓        5000
💕        5000
👅        4531
emoji       1
Length: 61, dtype: int64

Remove Duplicates

In [5]:
df.drop_duplicates(keep='first', subset="tweets",inplace=True)
df.info()
#df.loc[120].tweets
df=df.reset_index()

#data cleaning process
raw_dataset = df['tweets']
target_emoji = df['emoji']
tweet_id=df['id']
tweet_time=df['time']
#df.drop('index')
#print(raw_dataset)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283895 entries, 0 to 300517
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      283895 non-null  object
 1   time    283895 non-null  object
 2   tweets  283895 non-null  object
 3   emoji   283895 non-null  object
dtypes: object(4)
memory usage: 10.8+ MB


In [6]:
temp_list=df.index[df['emoji'] == '👅'].tolist()
print(len(temp_list))

4058


In [7]:
display(df)

,index,id,time,tweets,emoji
0,0,1593507164559056897,2022-11-18 07:31:28,@BLACKPINKSTATS5 Love to see it 👏 the numbers ...,👏
1,1,1593507163971911682,2022-11-18 07:31:28,@PostmanCarson congratulations John 🎊 👏,👏
2,2,1593507163506462721,2022-11-18 07:31:28,@Tyranjf @RunningWithTum1 @we_are_runners @Ich...,👏
3,3,1593507132296527873,2022-11-18 07:31:20,A story book scene.. congrats to both of you! 👏,👏
4,4,1593507130291769345,2022-11-18 07:31:20,@thatpositivepie 👏👏👏 and good look,👏
...,...,...,...,...,...
283890,300239,1593564433950834688,2022-11-18 11:19:02,@misssophyxo How sexy the body would lick you💋...,👅
283891,300435,1593541867089567745,2022-11-18 09:49:22,@similolamobee Mama I saw the caption \nI am n...,👅
283892,300440,1593541295082856448,2022-11-18 09:47:05,"A day later, I'm still looking at Mommy's musc...",👅
283893,300467,1593536193651695619,2022-11-18 09:26:49,👅Drop your handles let’s follow you!,👅


Remove hastags, urls, mentions, BOM character, none-English words and extra spaces after cleaning

In [ ]:
# import nltk 
# nltk.download('words')
# words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [33]:
#reference to https://github.com/giuliadnt/emo_tweepy with some changes
def clean_tweets(list_of_tweets):
    """
    :param list_of_tweets: list of unique tweets (no duplicates)
    :return: list of preprocessed tweets
    the function standardizes mentions --> @USER
    and URLs --> @URL
    removes BOM character
    """
    # matches @username
    # mentions = re.compile('(@.*?)(?=\s|\:)')#(^|\s)
    # links = re.compile('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
    # hashtags = re.compile('#([a-zA-Z0-9_]{1,50})')
    # L = re.sub('[0-9]+', '', row['tweets']) 
    # L=re.split('[ ,.!?:$&*/\|<>^+=-_-•%;。･，<>]',L)  
    #no_mentions = [re.sub(mentions, "", i) for i in list_of_tweets]
    punctuation=string.punctuation
    lower_case = [i.lower() for i in list_of_tweets]
    list_of_tweets=[clean(i,no_emoji=True) for i in lower_case] #remove in-text emojis
    lower_case = [i.encode('ascii', 'ignore').decode() for i in lower_case] # remove unicode characters
    # lower_case=[re.sub(punctuation, "", i) for i in lower_case]
    
    no_number=[re.sub('[0-9]+', "", i) for i in lower_case]
    no_number=[re.sub(r'\s[a-z]\s|\s[0-9]\s', " ", i) for i in no_number]
 
    no_mentions = [re.sub(r"@\S+", "", i) for i in no_number]
   
    no_links = [re.sub(r"http\S+", "", i) for i in no_mentions]
    no_links = [re.sub(r"www.\S+", "", i) for i in no_links]
    no_BOM = [re.sub(u"\ufeff", "", i) for i in no_links]
    #no_hash = [re.sub(hashtags,"",i)for i in no_BOM]
    no_hash = [re.sub(r"#\S+","",i)for i in no_BOM]
    no_newlinechar = [re.sub('\n', "", i) for i in no_hash]
    no_newlinechar=[i.translate(str.maketrans('', '', string.punctuation)) for i in no_newlinechar]
    #no_nonEnglish = [" ".join(w for w in nltk.wordpunct_tokenize(sent) if w.lower() in words or not w.isalpha()) for sent in no_newlinechar]
    # for sent in no_newlinechar:
    #   for word in sent:
    #     word = " ".join(w for w in nltk.wordpunct_tokenize(word) if w.lower() in words or not w.isalpha())
    no_extraspace = [" ".join(i.split()) for i in no_newlinechar]
    return no_extraspace

In [34]:
temp=["I need you girl 넌 아름다워 I! need you happy e girl👏 너무 차가워 I need you girl I need you girlI need you girl I need you girl - I NEED U -BTS Favorite K-Pop Artist","Yeeeeeeeeeeees.Happy Birthday @Tilly"]
clean_tweets(temp)
# sent = " ".join(w for w in nltk.wordpunct_tokenize(temp) if w.lower() in words or not w.isalpha())
# print(sent)

['i need you girl i need you happy girl need you girl need you girli need you girl need you girl need bts favorite kpop artist',
 'yeeeeeeeeeeeeshappy birthday']

In [35]:
temp=["A 你好story book scene.. congrats to both of you! 👏"]
print("Before clean:",temp)
print("After clean:",clean_tweets(temp))

Before clean: ['A 你好story book scene.. congrats to both of you! 👏']
After clean: ['a story book scene congrats to both of you']


In [32]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [36]:
#remove hashtags,urls,@,extraspances..
cleaned_dataset=clean_tweets(raw_dataset)

In [13]:
cleaned_dataset[0:10]

['blackpinkstats love to see it the numbers they both pull is mind blowing talent speaks for itself',
 'postmancarson congratulations john',
 'tyranjf runningwithtum wearerunners ichoosebactive well done champ',
 'a story book scene congrats to both of you',
 'thatpositivepie and good look',
 'rjayowo four arms more like for us',
 'cleanfeedttvr love the mogg',
 'here here an amazing team of reps delivering and making work better for our members',
 'enronchairman they should do that to all cex',
 'congratulations chi chi']

In [37]:
#organize post and emoji to a dataframe
data2 = {'id':tweet_id,
         'time':tweet_time,
         'tweets':cleaned_dataset,
        'emoji':target_emoji}

Emojimotion_df = pd.DataFrame(data2)

Drop the sentences that are less than 4 words

In [38]:
ind_list=[]
ind=0
temp=cleaned_dataset
for tw in temp:
  tw_word_list = re. findall(r"[\w'\"]+|[,.!?] ", tw)
  # print(tw_word_list)
  #print(len(tw_word))
  if len(tw_word_list)<4:
    ind_list.append(ind)
  ind=ind+1
Emojimotion_df.drop(ind_list,axis=0,inplace=True)
Emojimotion_df=Emojimotion_df.reset_index()
Emojimotion_df.head()

,index,id,time,tweets,emoji
0,0,1593507164559056897,2022-11-18 07:31:28,love to see it the numbers they both pull is m...,👏
1,3,1593507132296527873,2022-11-18 07:31:20,a story book scene congrats to both of you,👏
2,5,1593507124596084736,2022-11-18 07:31:19,four arms more like for us,👏
3,7,1593507111023034369,2022-11-18 07:31:15,here here an amazing team of reps delivering a...,👏
4,8,1593507062893641728,2022-11-18 07:31:04,they should do that to all cex,👏


In [39]:
Emojimotion_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212295 entries, 0 to 212294
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   index   212295 non-null  int64 
 1   id      212295 non-null  object
 2   time    212295 non-null  object
 3   tweets  212295 non-null  object
 4   emoji   212295 non-null  object
dtypes: int64(1), object(4)
memory usage: 8.1+ MB


In [69]:
#write the finnal dataframe to a csv file
Emojimotion_df.to_csv('cleaned_tweets.csv',index=False)

In [40]:
data = Emojimotion_df.groupby("emoji").size().sort_values(ascending = False)
data

emoji
➖     4563
💓     4453
👇     4245
💩     4237
😕     4218
🎧     4182
▶️    4167
🔴     4151
😬     4150
😠     4106
💰     4106
💀     4095
😡     4089
😓     4079
📍     4077
😔     4029
😣     4019
😫     4014
😞     4005
😝     3936
☁️    3900
😄     3870
😮     3869
❗️    3773
😳     3764
⚡️    3681
😹     3676
💛     3644
😌     3589
💃     3577
😃     3545
💙     3506
✊     3492
💖     3478
✅     3428
✔️    3367
🙏     3339
🌷     3338
😋     3260
😚     3194
🎁     3169
✨     3164
👑     3137
💕     3075
💗     3072
🌈     3063
💚     3050
😘     3044
💜     3037
💘     3015
💋     3014
🍃     2998
🌹     2809
😻     2781
🌺     2775
👅     2745
👌     2661
⬅️    2545
👏     2534
☕️    2396
dtype: int64